## Install Required Libraries

In [11]:
# install github branch feat/pytorch-detach from  git+https://github.com/gon-uri/detach_rocket

!pip install sktime --quiet
#!pip install  git+https://github.com/gon-uri/detach_rocket.git@feat/pytorch-detach --quiet
!pip install pyts --quiet
!pip install matplotlib --quiet
!pip install torch --quiet

## Download Dataset from UCR

In [80]:
import sys
sys.path.append('..')

from detach_rocket.utils_datasets import fetch_ucr_dataset

# Download Dataset
dataset_name_list = ['ProximalPhalanxOutlineAgeGroup'] # PhalangesOutlinesCorrect ProximalPhalanxOutlineCorrect #Fordb
current_dataset = fetch_ucr_dataset(dataset_name_list[0])

## Prepare Dataset Matrices

In [140]:
import numpy as np

# Create data matrices and remove possible rows with nans

print(f"Dataset Matrix Shape: ( # of instances , time series length )")
print(f" ")

# Train Matrix
X_train = current_dataset['data_train']

non_nan_mask_train = ~np.isnan(X_train).any(axis=1)
non_inf_mask_train = ~np.isinf(X_train).any(axis=1)
mask_train = np.logical_and(non_nan_mask_train,non_inf_mask_train)
X_train = X_train[mask_train]
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
y_train = current_dataset['target_train']
y_train = y_train[mask_train]
print(f"Train: {X_train.shape}")

print(f" ")

# Test Matrix
X_test = current_dataset['data_test']
#print(f"Number of test instances: {len(X_test)}")

non_nan_mask_test = ~np.isnan(X_test).any(axis=1)
non_inf_mask_test = ~np.isinf(X_test).any(axis=1)
mask_test = np.logical_and(non_nan_mask_test,non_inf_mask_test)
X_test = X_test[mask_test]
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
y_test = current_dataset['target_test']
y_test = y_test[mask_test]
print(f"Test: {X_test.shape}")

print(f" ")
# Number of classes
n_classes = len(np.unique(y_train))
print(f"Number of classes: {n_classes}")

# Convert y_train and y_test to start from 0
y_train = y_train - np.min(y_train)
y_test = y_test - np.min(y_test)

Dataset Matrix Shape: ( # of instances , time series length )
 
Train: (400, 1, 80)
 
Test: (205, 1, 80)
 
Number of classes: 3


## Train and Evaluate the Model

In [153]:
from detach_rocket.detach_classes import RocketFeaturesPytorch, RocketPytorch


# Import torch and early stopping
import torch

np.random.seed(2)

# Select initial model characteristics
c_in = X_train.shape[1]

# If binary classification, c_out = 1, else c_out = n_classes
if n_classes == 2:
    c_out = 1
else:
    c_out = n_classes

seq_len = X_train.shape[2]
num_kernels=1000
L2_regularization_weight = 1e-3

# Create model object
PytorchRocketModel = RocketPytorch(c_in,c_out,seq_len,num_kernels)
PytorchRocketModel = PytorchRocketModel.float()

# Define early stopping class
class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve after a given patience.
    """
    def __init__(self, patience=20, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 20
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score - self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else: 
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

# Create DataLoaders
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# Define batch size
batch_size = 2048

# Define train function for Pytorch model with epochs and early stopping
def train_model(model, train_data, test_data, batch_size, patience):
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    # Create DataLoaders
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=False)
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=False)
    # Define loss function and optimizer (for multi-class classification)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=L2_regularization_weight)
    # Define early stopping
    #early_stopping = EarlyStopping(patience=patience, verbose=True)
    # Define number of epochs
    n_epochs = 100
    # Initialize lists for train and test losses
    train_losses = []
    test_losses = []
    # Train model
    for epoch in range(n_epochs):
        # Initialize variables to monitor training and test loss
        train_loss = 0.0
        test_loss = 0.0
        # Train model
        model.train()
        for data, target in train_loader:
            # Clear the gradients of all optimized variables
            optimizer.zero_grad()
            # Forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float())
            # Calculate the batch loss
            target = target.long()
            # If not multi-class classification, target needs to be reshaped
            if c_out == 1:
                target = target.unsqueeze(1)
            #print('output: ',output.shape)
            #print('target: ',target.shape)
            loss = criterion(output, target)
            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Perform a single optimization step (parameter update)
            optimizer.step()
            # Update training loss
            train_loss += loss.item()*data.size(0)
        # Validate model
        model.eval()
        for data, target in test_loader:
            # Forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float())
            # Calculate the batch loss
            target = target.long()
            if c_out == 1:
                target = target.unsqueeze(1)
            loss = criterion(output, target)
            # Update average test loss 
            test_loss += loss.item()*data.size(0)
        # Calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        test_loss = test_loss/len(test_loader.dataset)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        # Print training and test results
        print(f'Epoch: {epoch+1} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {test_loss:.6f}')
        # Early stop
        early_stopping(test_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

[7, 9, 11]
2000


In [154]:
# Train model
train_model(PytorchRocketModel, train_data, test_data, batch_size, patience=20)

Epoch: 1 	Training Loss: 1.098612 	Validation Loss: 2.045879
Validation loss decreased (inf --> 2.045879).  Saving model ...
Epoch: 2 	Training Loss: 0.619119 	Validation Loss: 2.745318
EarlyStopping counter: 1 out of 20
Epoch: 3 	Training Loss: 0.621798 	Validation Loss: 3.048682
EarlyStopping counter: 2 out of 20
Epoch: 4 	Training Loss: 0.653161 	Validation Loss: 2.957419
EarlyStopping counter: 3 out of 20
Epoch: 5 	Training Loss: 0.649564 	Validation Loss: 2.614972
EarlyStopping counter: 4 out of 20
Epoch: 6 	Training Loss: 0.612249 	Validation Loss: 2.140547
EarlyStopping counter: 5 out of 20
Epoch: 7 	Training Loss: 0.551696 	Validation Loss: 1.628092
Validation loss decreased (2.045879 --> 1.628092).  Saving model ...
Epoch: 8 	Training Loss: 0.479001 	Validation Loss: 1.163137
Validation loss decreased (1.628092 --> 1.163137).  Saving model ...
Epoch: 9 	Training Loss: 0.412116 	Validation Loss: 0.829146
Validation loss decreased (1.163137 --> 0.829146).  Saving model ...
Epoch

In [161]:
# Load best model
PytorchRocketModel.load_state_dict(torch.load('checkpoint.pt'))

# Compute model accuracy both on train and test set
from sklearn.metrics import accuracy_score

# Create DataLoaders
train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size, drop_last=False)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=False)

# Initialize lists for train and test predictions
train_predictions = []
test_predictions = []

# Compute train predictions
PytorchRocketModel.eval()
# If it is a binary classification problem, convert probabilities to class labels
if len(np.unique(y_train)) == 2:
    for data, target in train_loader:
        output = PytorchRocketModel(data.float())
        output = torch.sigmoid(output)
        output = output.detach().numpy()
        # Convert probabilities to class labels
        output = np.where(output > 0.5, 1, 0)
        train_predictions.extend(output)
# If it is a multiclass classification problem, select class with maximum probability
else:
    for data, target in train_loader:
        output = PytorchRocketModel(data.float())
        output = torch.sigmoid(output)
        output = output.detach().numpy()
        # Select class with maximum probability
        output = np.argmax(output, axis=1)
        # Convert probabilities to class labels
        train_predictions.extend(output)

# Compute test predictions 
PytorchRocketModel.eval()
# If it is a binary classification problem, convert probabilities to class labels
if len(np.unique(y_train)) == 2:
    for data, target in test_loader:
        output = PytorchRocketModel(data.float())
        output = torch.sigmoid(output)
        output = output.detach().numpy()
        # Convert probabilities to class labels
        output = np.where(output > 0.5, 1, 0)
        test_predictions.extend(output)
# If it is a multiclass classification problem, select class with maximum probability
else:
    for data, target in test_loader:
        output = PytorchRocketModel(data.float())
        output = torch.sigmoid(output)
        output = output.detach().numpy()
        # Select class with maximum probability
        output = np.argmax(output, axis=1)
        # Convert probabilities to class labels
        test_predictions.extend(output)

# Compute accuracy
train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f"Train accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")

Train accuracy: 0.9275
Test accuracy: 0.8536585365853658


In [215]:
def get_feature_importance(model, multilabel_type="max"):
    """
    Compute feature importance of a trained model.

    Parameters:
    - model: Trained model.
    - multilabel_type: Type of feature ranking in case of multilabel classification ("max" by default).

    Returns:
    - feature_importance: Feature importance vector.
    """
    
    # Get last layer weights
    last_layer_weights = model.head[2].weight.detach().numpy()

    # Check if it is a binary or multiclass classification problem looking at number of dimensions of last layer weights
    if len(last_layer_weights.shape) > 1:
        if multilabel_type == "norm":
            feature_importance   = np.linalg.norm(last_layer_weights,axis=0,ord=2)
        elif multilabel_type == "max":
            feature_importance  = np.linalg.norm(last_layer_weights,axis=0,ord=np.inf)
        elif multilabel_type == "avg":
            feature_importance= np.linalg.norm(last_layer_weights,axis=0,ord=1)
        else:
            raise ValueError('Invalid multilabel_type argument. Choose from: "norm", "max", or "avg".')
    else:
        feature_importance = np.abs(last_layer_weights)
    return feature_importance

In [220]:
# Compute feature importance
feature_importance = get_feature_importance(PytorchRocketModel, multilabel_type="max")

# Compute number of features
n_features = len(feature_importance)

# Percentage of features to drop
drop_percentage = 0.05

# Compute number of features to drop
n_features_to_drop = int(np.floor(drop_percentage*n_features))

# Create mask of features to drop
feature_mask = np.zeros(len(feature_importance), dtype=bool)
feature_mask[np.argsort(feature_importance)[:n_features_to_drop]] = True

In [225]:
# Create a weigth mask
weight_mask = np.zeros((c_out,len(feature_importance)), dtype=bool)
for i in range(c_out):
    weight_mask[i,:] = feature_mask

# Create weight matrix sending to zero weights of dropped features
weights_matrix = np.ones((c_out,len(feature_importance)))
weights_matrix[weight_mask] = 0
weights_matrix = np.multiply(PytorchRocketModel.head[2].weight.detach().numpy(),weights_matrix)

# Print amount of zeros in weight matrix
print(f"Percentage of zeros in weight matrix: {np.sum(weights_matrix == 0)/(c_out*n_features)} %")

# Convert weights_matrix to torch torch.nn.parameter 
weights_matrix = torch.from_numpy(weights_matrix)
weights_matrix = torch.nn.Parameter(weights_matrix.float())

PytorchRocketModel.head[2].weight = weights_matrix

# ACA ESTA EL ERROR
# Esto no anda, no se puede poner requires_grad para cada peso
PytorchRocketModel.head[2].weight.requires_grad[~weight_mask] = False


Percentage of zeros in weight matrix: 0.05 %


TypeError: 'bool' object does not support item assignment

In [226]:
PytorchRocketModel.head[2].weight.requires_grad

True

In [209]:
PytorchRocketModel.head[2].weight.shape

torch.Size([3, 2000])

(1, 2000)

In [193]:
# Esto lo habia empezado a escribir para cuando haya que selencionar kernels a partir de features

# Create a kernel mask
kernel_mask_list = []
for i in range(int(n_features/2)):
    # If two consecutive features are dropped, drop the corresponding kernel
    if feature_mask[2*i] == False and feature_mask[2*i+1] == False:
        kernel_mask_list.append(False)
    else:
        kernel_mask_list.append(True)

# Drop kernels
